# Health Study – Statistical Analysis Report

## Summary
This notebook presents a complete statistical analysis of a health study dataset.
The aim is to explore basic characteristics of the sample, quantify uncertainty, and
evaluate a specific hypothesis about whether smokers have higher systolic blood pressure
than non-smokers. The analysis includes:

- Data loading and initial inspection
- Descriptive statistics for key health variables
- Visualizations of distributions and group differences
- Estimation of disease prevalence and a simple simulation based on the data
- Confidence intervals for systolic blood pressure using two different methods
- Hypothesis testing if smoking has any effect on the blood pressure
- A power (sensitivity) analysis of the hypothesis test


## Setup,imports and inspection

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

np.random.seed(42)

REQUIRED = ["age", "weight", "height", "systolic_bp", "cholesterol"]

df = pd.read_csv("data/health_study_dataset.csv")

missing = [c for c in REQUIRED if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")
else:
    print("All required columns are present.")

df.head()

## Data Loading Helper

In [ ]:
def f_load_csv(path="Data/health_study_dataset.csv"):
    return pd.read_csv(path)

df = f_load_csv()


## Statistical Utility Functions

In [ ]:
def descriptive_stats(df: pd.DataFrame, cols):                      ## Returns mean, median, min & max for selected columns
        return df[cols].agg(["mean", "median", "min", "max"]).T


def ci_mean_normal_approx(data, alpha: float = 0.05):               ## Using normal approx -> confidence interval for the mean
    data = np.asarray(data)
    n = len(data)
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    z = stats.norm.ppf(1 - alpha / 2)
    se = std / np.sqrt(n)
    return mean, (mean - z * se, mean + z * se)


def ci_mean_bootstrap(data, alpha: float = 0.05, n_boot: int = 5000, random_state: int | None = None):
    rng = np.random.default_rng(random_state)                       ## With percentile method-> bootstrap confidence interval for the mean
    data = np.asarray(data)
    n = len(data)
    boot_means = np.empty(n_boot)
    for i in range(n_boot):
        sample = rng.choice(data, size=n, replace=True)
        boot_means[i] = np.mean(sample)
    lower = np.percentile(boot_means, 100 * (alpha / 2))
    upper = np.percentile(boot_means, 100 * (1 - alpha / 2))
    return np.mean(data), (lower, upper), boot_means


def ttest_smokers_bp(smokers_bp, nonsmokers_bp, alpha: float = 0.05):   
    
    smokers_bp = np.asarray(smokers_bp)                             ## Welch-test, one sided: H1= smokers have higher BP mean
    nonsmokers_bp = np.asarray(nonsmokers_bp)
    t_stat, p_two = stats.ttest_ind(smokers_bp, nonsmokers_bp, equal_var=False)
    # convert to one-sided p-value
    if t_stat > 0:
        p_one = p_two / 2
    else:
        p_one = 1 - p_two / 2
    return {
        "t_stat": t_stat,
        "p_value": p_one,
        "reject_H0": p_one < alpha,
    }


def simulate_disease(n_sim: int = 1000, p: float = 0.1, random_state: int | None = None):
    
    rng = np.random.default_rng(random_state)                   ## Simulate n_sin Bernoulli trials with probability p (disease)
    sim_data = rng.binomial(1, p, size=n_sim)
    return sim_data.mean(), sim_data


def power_simulation_ttest(smokers_bp, nonsmokers_bp, n_sim: int = 500, alpha: float = 0.05, random_state: int | None = None):
   
    rng = np.random.default_rng(random_state)                   ## Estimated power of the one sided t-test by resampling from the observed data
    smokers_bp = np.asarray(smokers_bp)
    nonsmokers_bp = np.asarray(nonsmokers_bp)
    n_s = len(smokers_bp)
    n_ns = len(nonsmokers_bp)
    rejections = 0

    for _ in range(n_sim):
        s_sample = rng.choice(smokers_bp, size=n_s, replace=True)
        ns_sample = rng.choice(nonsmokers_bp, size=n_ns, replace=True)
        t_stat, p_two = stats.ttest_ind(s_sample, ns_sample, equal_var=False)
        if t_stat > 0:
            p_one = p_two / 2
        else:
            p_one = 1 - p_two / 2
        if p_one < alpha:
            rejections += 1

    return rejections / n_sim


## Visualization Functions

In [ ]:
def plot_histogram(df: pd.DataFrame, column: str, bins: int = 20):
    plt.figure(figsize=(8, 5))
    plt.hist(df[column], bins=bins, edgecolor="black")
    plt.xlabel(column)
    plt.ylabel("Count")
    plt.title(f"Histogram of {column}")
    plt.show()


def plot_box_by_group(df: pd.DataFrame, column: str, group: str):
    plt.figure(figsize=(8, 5))
    df.boxplot(column=column, by=group)
    plt.xlabel(group)
    plt.ylabel(column)
    plt.title(f"Boxplot of {column} by {group}")
    plt.suptitle("")
    plt.show()


def plot_bar_share(df: pd.DataFrame, column: str):
    plt.figure(figsize=(7, 5))
    df[column].value_counts(normalize=True).plot(kind="bar", edgecolor="black")
    plt.ylabel("Proportion")
    plt.title(f"Share of {column}")
    plt.xticks(rotation=0)
    plt.show()


## Descriptive Statistics

In [ ]:
stats_table = descriptive_stats(df, ['age', 'weight', 'height', 'systolic_bp', 'cholesterol'])
stats_table

**Interpretation of the descriptive statistics:**

The table summarizes the central tendency and range of the key health variables.

Mean and median provide insight into typical values, while the minimum and maximum highlight the spread of the data.

Large differences between mean and median or noticeably wide ranges may signal skewness or the presence of outliers.

## Visualizing Key Variables

In [ ]:
plot_histogram(df, 'systolic_bp')

**Histogram interpretation:**

The histogram illustrates how systolic blood pressure values are distributed across the sample.
A roughly symmetric shape suggests approximate normality, while visible skewness or multiple peaks 
could indicate underlying subgroups, measurement differences, or variability in the population.

In [ ]:
plot_box_by_group(df, 'weight', 'sex')

**Boxplot interpretation:**
The boxplot compares the distribution of body weight between sexes.

Differences in median values, interquartile ranges, or the presence of outliers 
may reveal systematic differences between groups or greater heterogeneity within one group.

In [ ]:
plot_bar_share(df, 'smoker')

**Bar chart imterpretation:**

The bar chart shows the proportion of smokers and non-smokers in the dataset.

The relative sizes of these groups are important when interpreting the hypothesis,
test and evaluating how well each subgroup is represented.

## Disease Prevalence and Simulation

In [ ]:
true_rate = df['disease'].mean()
sim_rate, sim_data = simulate_disease(n_sim=1000, p=true_rate, random_state=42)
true_rate, sim_rate

**Interpretation:**

The observed disease prevalence in the dataset is approx. 5.88%.
Simulating 1,000 individuals using this same probability produced a prevalence of 5.4%.

These values are very similar, which is expected due to random variation in simulation.

This agreement indicates that a simple Bernoulli model effectively captures the underlying probability structure of the disease variable.

## Confidence Intervals for Mean Systolic Blood Pressure

In [ ]:
bp = df['systolic_bp'].values
ci_norm = ci_mean_normal_approx(bp)
ci_boot = ci_mean_bootstrap(bp, random_state=42)[:2]
ci_norm, ci_boot

**Interpretation:**

Both methods estimate the mean systolic blood pressure to be approximately 149.18 mmHg.

Normal approximation:    95% CI: [148.29, 150.07]
Bootstrap:               95% CI: [148.30, 150.06]

The two intervals almost perfectly overlap, indicating:

- The sampling distribution of the mean is close to normal
- The dataset is sufficiently large for the Central Limit Theorem to apply
- No major skewness or irregularities appear in the bootstrap distribution
- The mean estimate is highly stable and robust
- The near-identical intervals reinforce confidence in the precision of the estimated mean systolic blood pressure.

## Hypothesis Test: Do Smokers Have Higher Systolic BP?

In [ ]:
smokers_bp = df[df['smoker'] == 'Yes']['systolic_bp']
nonsmokers_bp = df[df['smoker'] == 'No']['systolic_bp']
test_results = ttest_smokers_bp(smokers_bp, nonsmokers_bp)
test_results

**Interpretation:**

The one-sided Welch's t-test evaluates the hypothesis:

- H0: Smokers and non-smokers have the same mean systolic BP
- H1: Smokers have higher mean systolic BP than non-smokers

A small p-value (typically < 0.05) provides evidence against H0 in favour of H1.
If the test rejects H0, we conclude that smokers in this sample tend to have
higher systolic blood pressure in this sample.
If not, we cannot rule out that any observed difference is due to random variation.

## Power Analysis of the t-test

In [ ]:
power_estimate = power_simulation_ttest(smokers_bp, nonsmokers_bp, n_sim=500, random_state=42)
power_estimate

**Interpretation:**

The estimated power indicates how often the test correctly detects a true difference in mean systolic blood pressure between smokers and non-smokers, assuming the dataset reflects the population.

Power values above 0.8 are typically considered strong
Lower power implies a higher risk of Type II error (failing to detect a real difference)

Power analysis therefore provides essential context:

A significant test result with high power is convincing
A non-significant test with low power should be interpreted with caution
“The estimated power was 0.088

## Conclusion

This analysis provides a clear overview of the health dataset and applies several complementary statistical methods. The estimates of mean systolic blood pressure were highly consistent between the normal approximation and the bootstrap approach, indicating that the mean is stable and that the data conform well to the assumptions behind both methods.

The hypothesis test found no statistically significant evidence that smokers have higher systolic blood pressure than non-smokers. However, the difference in group means was very small, and the power analysis showed that the test had low sensitivity. This means that even if a true difference existed, the current sample may be too limited to reliably detect it.

Overall, the results do not support a meaningful difference in systolic blood pressure between smokers and non-smokers in this dataset. At the same time, the low statistical power suggests that larger samples or stronger effect sizes would be necessary to draw firmer conclusions. Despite these limitations, the combination of descriptive statistics, resampling methods, and simulation provides a solid and transparent analytical foundation.

While smoking shows mixed evidence regarding long-term effects on blood pressure, several Scandinavian studies suggest that habitual snus use does not appear to elevate chronic blood pressure.
In other words—strictly from a blood-pressure standpoint—you might say that switching from smoking to snus is at least an upgrade.


## References

Efron, B., & Tibshirani, R. J. (1994). An Introduction to the Bootstrap. Chapman & Hall.

Mooney, C. Z., & Duval, R. D. (1993). Bootstrapping: A Nonparametric Approach to Statistical Inference. Sage.

Student. (1908). The probable error of a mean. Biometrika, 6(1), 1–25.

Cohen, J. (1988). Statistical Power Analysis for the Behavioral Sciences (2nd ed.). Lawrence Erlbaum Associates.

Wasserman, L. (2004). All of Statistics: A Concise Course in Statistical Inference. Springer.

Altman, D. G., & Bland, J. M. (1995). Statistics notes: The normal distribution. BMJ, 310, 298.

Freedman, D., Pisani, R., & Purves, R. (2007). Statistics (4th ed.). W. W. Norton & Company.

